# HC01 — Healthcare Claims EDA Notebook
Author: Bhanu Garg  
Created: 2025-09-03 03:08:40

_Goal: Load claims, members, and providers data; perform basic cleaning; derive useful columns; join; explore; and visualize._

## 1) Setup & Imports

In [ ]:

# Do NOT use seaborn. We stick to matplotlib as per project conventions.
import os, sys, math, json, textwrap
from datetime import datetime, date

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Make pandas output easier to read
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 120)

print('Python', sys.version)
print('Pandas', pd.__version__)
print('NumPy', np.__version__)


## 2) Configure File Paths

In [ ]:

# Adjust paths if your repo structure differs
DATA_DIR = '../data'  # if this notebook lives in notebooks/
CLAIMS_CSV = os.path.join(DATA_DIR, 'claims.csv')
MEMBERS_CSV = os.path.join(DATA_DIR, 'members.csv')
PROVIDERS_CSV = os.path.join(DATA_DIR, 'providers.csv')

for p in [CLAIMS_CSV, MEMBERS_CSV, PROVIDERS_CSV]:
    print(p, 'exists:', os.path.exists(p))


## 3) Load CSVs

In [ ]:

def try_read_csv(path, parse_date_cols=None):
    parse_date_cols = parse_date_cols or []
    try:
        df = pd.read_csv(path, parse_dates=[c for c in parse_date_cols if c in pd.read_csv(path, nrows=1).columns])
        print(f'✅ Loaded: {path} -> shape={df.shape}')
        return df
    except FileNotFoundError:
        print(f'⚠️ File not found: {path}')
        return pd.DataFrame()
    except Exception as e:
        print(f'⚠️ Error reading {path}: {e}')
        return pd.DataFrame()

claims  = try_read_csv(CLAIMS_CSV, parse_date_cols=['service_date', 'claim_date', 'paid_date'])
members = try_read_csv(MEMBERS_CSV, parse_date_cols=['dob'])
providers = try_read_csv(PROVIDERS_CSV)

def peek(df, name):
    print(f'\n{name} head:')
    display(df.head(3))
    print(f'{name} dtypes:')
    print(df.dtypes)
    print(f'{name} columns:', list(df.columns))

peek(claims, 'claims')
peek(members, 'members')
peek(providers, 'providers')


## 4) Basic Cleaning

In [ ]:

def standardize_columns(df):
    # Strip, lower, replace spaces with underscores
    df.columns = [c.strip().lower().replace(' ', '_') for c in df.columns]
    return df

claims = standardize_columns(claims)
members = standardize_columns(members)
providers = standardize_columns(providers)

# Deduplicate by common IDs if present
for df, name, key in [(claims, 'claims', 'claim_id'), (members, 'members', 'member_id'), (providers, 'providers', 'provider_id')]:
    if key in df.columns:
        before = len(df)
        df.drop_duplicates(subset=[key], inplace=True)
        after = len(df)
        if before != after:
            print(f'ℹ️ Deduped {name} on {key}: {before} -> {after}')

# Handle obvious empty strings as NaN
for df in [claims, members, providers]:
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].replace({'': np.nan, 'NA': np.nan, 'N/A': np.nan})

print('✅ Basic cleaning complete.')


## 5) Derive Columns

In [ ]:

# year_month from service_date/claim_date if available
def add_year_month(df, date_cols=('service_date','claim_date')):
    for c in date_cols:
        if c in df.columns and np.issubdtype(df[c].dtype, np.datetime64):
            df['year_month'] = df[c].dt.to_period('M').astype(str)
            print(f'✅ Derived year_month from {c}')
            return df
    print('⚠️ No datetime column found among', date_cols, '— skipped year_month.')
    return df

def compute_age_years(members_df, asof_col=None):
    # If dob and as-of date available (e.g., claim service_date), compute age
    if 'dob' not in members_df.columns:
        print('⚠️ members.dob missing — skipping age computation.')
        return members_df
    # If asof_col is supplied and exists, use it post-join; otherwise skip for now.
    members_df['birth_year'] = members_df['dob'].dt.year
    return members_df

claims = add_year_month(claims)
members = compute_age_years(members)

# Normalize dollar amounts if present
for amt_col in ['billed_amount','allowed_amount','paid_amount']:
    if amt_col in claims.columns:
        # Remove currency symbols/commas
        if claims[amt_col].dtype == object:
            claims[amt_col] = claims[amt_col].str.replace('[\$,]', '', regex=True)
        claims[amt_col] = pd.to_numeric(claims[amt_col], errors='coerce')

print('✅ Derivations done.')


## 6) Join Tables

In [ ]:

# Attempt joins if keys exist
joined = claims.copy()
join_keys = []

if 'member_id' in claims.columns and 'member_id' in members.columns:
    joined = joined.merge(members.add_prefix('m_'), left_on='member_id', right_on='m_member_id', how='left')
    join_keys.append('member_id')
if 'provider_id' in claims.columns and 'provider_id' in providers.columns:
    joined = joined.merge(providers.add_prefix('p_'), left_on='provider_id', right_on='p_provider_id', how='left')
    join_keys.append('provider_id')

print('Joined on keys:', join_keys if join_keys else 'none (using claims only)')
display(joined.head(3))
print('joined shape:', joined.shape)


## 7) Summary Statistics

In [ ]:

def summarize(df):
    out = {}
    out['n_rows'] = len(df)
    for col in ['claim_id', 'member_id', 'provider_id']:
        if col in df.columns:
            out[f'n_unique_{col}'] = df[col].nunique()
    if 'paid_amount' in df.columns:
        out['total_paid'] = float(np.nansum(df['paid_amount'].values))
        out['avg_paid'] = float(np.nanmean(df['paid_amount'].values))
    if 'billed_amount' in df.columns:
        out['total_billed'] = float(np.nansum(df['billed_amount'].values))
    if 'status' in df.columns:
        # assuming values like 'Denied', 'Paid', etc.
        denom = len(df[df['status'].notna()])
        if denom > 0:
            out['denial_rate'] = float((df['status'].astype(str).str.lower().eq('denied')).mean())
    return pd.Series(out)

summary_claims = summarize(claims)
summary_joined = summarize(joined)

print('Claims summary:')
display(summary_claims.to_frame('value'))
print('Joined summary:')
display(summary_joined.to_frame('value'))

# Save a quick profile
profile = pd.concat([summary_claims.rename('claims'), summary_joined.rename('joined')], axis=1)
profile_path = '../artifacts/eda_profile_summary.csv'
os.makedirs(os.path.dirname(profile_path), exist_ok=True)
profile.to_csv(profile_path)
print('📄 Saved profile to', profile_path)


## 8) Visuals (matplotlib only)

In [ ]:

# 8.1 Distribution of paid amount
if 'paid_amount' in claims.columns:
    plt.figure()
    claims['paid_amount'].dropna().plot(kind='hist', bins=50, alpha=0.7)
    plt.title('Distribution of Paid Amount')
    plt.xlabel('paid_amount')
    plt.ylabel('frequency')
    plt.show()
else:
    print('⚠️ paid_amount not found — skipping histogram.')


In [ ]:

# 8.2 Monthly paid trend
if 'year_month' in claims.columns and 'paid_amount' in claims.columns:
    monthly = claims.groupby('year_month', as_index=False)['paid_amount'].sum().sort_values('year_month')
    plt.figure()
    plt.plot(monthly['year_month'], monthly['paid_amount'])
    plt.title('Monthly Total Paid')
    plt.xlabel('year_month')
    plt.ylabel('total_paid')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print('⚠️ year_month or paid_amount missing — skipping monthly trend.')


In [ ]:

# 8.3 Top providers by total paid
provider_name_col = None
for cand in ['provider_name','name','p_provider_name','p_name']:
    if cand in joined.columns:
        provider_name_col = cand
        break

if provider_name_col and 'paid_amount' in joined.columns:
    topN = joined.groupby(provider_name_col, as_index=False)['paid_amount'].sum().sort_values('paid_amount', ascending=False).head(10)
    plt.figure()
    plt.bar(topN[provider_name_col], topN['paid_amount'])
    plt.title('Top 10 Providers by Total Paid')
    plt.xlabel('provider')
    plt.ylabel('total_paid')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print('⚠️ provider name column or paid_amount missing — skipping top providers chart.')


In [ ]:

# 8.4 Denial rate by month (if status + year_month present)
if 'status' in claims.columns and 'year_month' in claims.columns:
    tmp = claims.copy()
    tmp['is_denied'] = tmp['status'].astype(str).str.lower().eq('denied').astype(int)
    dr = tmp.groupby('year_month', as_index=False)['is_denied'].mean()
    plt.figure()
    plt.plot(dr['year_month'], dr['is_denied'])
    plt.title('Denial Rate by Month')
    plt.xlabel('year_month')
    plt.ylabel('denial_rate')
    plt.xticks(rotation=45)
    plt.ylim(0, 1)
    plt.tight_layout()
    plt.show()
else:
    print('⚠️ status or year_month missing — skipping denial rate chart.')


## 9) Next Steps & TODOs

- [ ] Validate column names in your CSVs and update key names if needed (`claim_id`, `member_id`, `provider_id`, `service_date`, `paid_amount`, etc.).
- [ ] Add a proper data dictionary under `../docs/data_dictionary.md`.
- [ ] Explore cohorts (e.g., member age bands vs. denial rate).
- [ ] Add unit tests for simple data quality checks (row counts, null rates).
- [ ] Capture screenshots of key charts for the README.
